## TUM RGB-D
---
intrinsic camera parameter
```markdown
fx = 525.0  # focal length x
fy = 525.0  # focal length y
cx = 319.5  # optical center x
cy = 239.5  # optical center y
```
RGB, Depth image로부터 Point cloud data 생성


issue: depth, rgb 데이터간 timestamp가 일치하지 않기 때문에 가장 가까운 시간으로 pairing 작업이 필요한 것 같다

In [12]:
import numpy as np
import cv2, os

In [14]:
# Camera intrinsic parameters
fx = 525.0 # focal length in x-direction
fy = 525.0 # focal length in y-direction
cx = 319.5 # principal point x-coordinate
cy = 239.5 # principal point y-coordinate

In [13]:
def read_association_file(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            if not line.startswith('#'):
                timestamp, filename = line.strip().split()
                data.append((float(timestamp), filename))
    return data

def depth_to_3d(depth, fx, fy, cx, cy):
    height, width = depth.shape
    u = np.linspace(0, width-1, width)
    v = np.linspace(0, height-1, height)
    u,v = np.meshgrid(u,v)
    
    X = (u - cx) * depth / fx
    Y = (v - cy) * depth / fy
    Z = depth
    
    return np.stack((X,Y,Z), axis=-1).reshape(-1, 3)

In [ ]:
base_dir = 'path_to_dataset_directory'  # Update this with the path to your dataset

depth_associations = read_association_file(os.path.join(base_dir, 'depth.txt'))
rgb_associations = read_association_file(os.path.join(base_dir, 'rgb.txt'))

for depth_timestamp, depth_filename in depth_associations:
    depth_image_path = os.path.join(base_dir, depth_filename)

    # Finding the closest RGB timestamp and filename
    closest_rgb_timestamp, closest_rgb_filename = min(rgb_associations, key=lambda x: abs(x[0] - depth_timestamp))
    rgb_image_path = os.path.join(base_dir, closest_rgb_filename)

    depth = cv2.imread(depth_image_path, -1) * 0.0002  # TUM depth images are saved in 16-bit with 5cm baseline, so multiplying by 0.0002 converts to meters
    rgb = cv2.imread(rgb_image_path)

    points_3d = depth_to_3d(depth, fx, fy, cx, cy)
    colors = rgb.reshape(-1, 3)

    # Continue with your processing...


In [16]:
## get closet timestamp rgb image
depth_timestamp = 1305031102.160407
depth_filename = '1305031102.160407.png'

base_dir = r"C:\Users\hojun\git\data\TUM_rgbd_dataset_freiburg1_xyz"
# Update this with the path to your dataset

rgb_associations = read_association_file(os.path.join(base_dir, 'rgb.txt'))

depth_image_path = os.path.join(base_dir, depth_filename)

# Finding the closest RGB timestamp and filename
closest_rgb_timestamp, closest_rgb_filename = min(rgb_associations, key=lambda x: abs(x[0] - depth_timestamp))
rgb_image_path = os.path.join(base_dir, closest_rgb_filename)

print(rgb_image_path)

C:\Users\hojun\git\data\TUM_rgbd_dataset_freiburg1_xyz\rgb/1305031102.175304.png


In [ ]:
depth_image = np.transpose(depths[0,:,:], (1, 0))
rgb_image = np.transpose(rgbs[0,:,:,:], (2, 1, 0))  # Transposing to have shape (width, height, 3)

point_cloud_rgb = depth_to_pointcloud_with_rgb(depth_image, rgb_image)

# Save to .csv format
np.savetxt(f'../data/nyu_depth_v2_labeled_csv/output_{2}.csv', point_cloud_rgb, delimiter=',', header='x,y,z,r,g,b', comments='', fmt='%f,%f,%f,%d,%d,%d')